# Organizing DF for COVIDxCT

In [ ]:
import pandas as pd
from sklearn.utils import resample

train_df = pd.read_csv('/media/riccelli/Disco 1/datasets_covid/covidxct/train_COVIDx_CT-3A.txt', sep=" ", header=None)
train_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
train_df=train_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )
#train_df = pd.read_csv('/media/riccelli/Disco 1/datasets_covid/covidxct/train_COVIDx_CT-3A_segmented.txt', sep=" ", header=None)
#train_df.columns=['filename', 'label']

# 读取test.txt
val_df = pd.read_csv('/media/riccelli/Disco 1/datasets_covid/covidxct/val_COVIDx_CT-3A.txt', sep=" ", header=None)
val_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
val_df=val_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

#test_df = pd.read_csv('/media/riccelli/Disco 1/datasets_covid/covidxct/test_COVIDx_CT-3A_segmented2.txt', sep=" ", header=None)
test_df = pd.read_csv('/media/riccelli/Disco 1/datasets_covid/covidxct/test_COVIDx_CT-3A.txt', sep=" ", header=None)
test_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
#test_df.columns=['filename', 'label']
test_df=test_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

#train_df = train_df.drop(train_df[train_df['filename'].str.contains("COVIDCTMD")].index)

# covidctmd_val = val_df[val_df['filename'].str.contains("COVIDCTMD")]
# covidctmd_val = val_df[val_df['label']==0]
# train_df = pd.concat([train_df, covidctmd_val])
# val_df = val_df.drop(val_df[val_df['filename'].str.contains("COVIDCTMD")].index)
val_df = val_df.drop(val_df[val_df['label']==0].index)

# covidctmd_test = test_df[test_df['filename'].str.contains("COVIDCTMD")]
# covidctmd_test = test_df[test_df['label']==0]
# train_df = pd.concat([train_df, covidctmd_test])
# test_df = test_df.drop(test_df[test_df['filename'].str.contains("COVIDCTMD")].index)
test_df = test_df.drop(test_df[test_df['label']==0].index)

image_path = '/media/riccelli/Disco 1/datasets_covid/covidxct/3A_images/'  #directory path
image_path2 = '/media/riccelli/Disco 1/datasets_covid/covidxct/3A_images_segmented/'  #directory path
image_path3 = '/media/riccelli/Disco 1/datasets_covid/covidxct/3A_test_images_segmented2/'  #directory path
# train_df['filename'] = image_path2+train_df['filename']
# val_df['filename'] = image_path+val_df['filename']
# test_df['filename'] = image_path3 + test_df['filename']
train_df['filename'] = image_path+train_df['filename']
val_df['filename'] = image_path+val_df['filename']
test_df['filename'] = image_path + test_df['filename']


# undersample
#N  = train_df[train_df['label']==0]
P = train_df[train_df['label']==1]
C = train_df[train_df['label']==2]
#N_download = resample(N, replace = True, n_samples = 25496,random_state=0)
#P_download = resample(P, replace = True, n_samples = 5000,random_state=0)
#C_download = resample(C, replace = True, n_samples = 5000,random_state=0)
#P_download = resample(P, replace = True, n_samples = 2000,random_state=0)
#C_download = resample(C, replace = True, n_samples = 1000,random_state=0)
#train_df = pd.concat([P_download, C_download])

# N_v  = val_df[val_df['label']==0]
# P_v = val_df[val_df['label']==1]
# C_v = val_df[val_df['label']==2]
# N_v_download = resample(N_v, replace = True, n_samples = 6244,random_state=0)
# #C_v_download = resample(C_v, replace = True, n_samples = 8008,random_state=0)
# P_v_download = resample(P_v, replace = True, n_samples = 6244,random_state=0)
# val_df = pd.concat([N_v_download, C_v, P_v_download])

train_df.loc[train_df["label"] == 1, "label"] = 0
train_df.loc[train_df["label"] == 2, "label"] = 1

val_df.loc[val_df["label"] == 1, "label"] = 0
val_df.loc[val_df["label"] == 2, "label"] = 1

test_df.loc[test_df["label"] == 1, "label"] = 0
test_df.loc[test_df["label"] == 2, "label"] = 1

labels={0:'Pneumonia',1:'COVID-19'}
class_names=['Pneumonia','COVID-19']

train_df['label_n']=[labels[b] for b in train_df['label']]
val_df['label_n']=[labels[b] for b in val_df['label']]
test_df['label_n']=[labels[b] for b in test_df['label']]

In [ ]:
print(f"Negative and positive values of train: \n{train_df['label_n'].value_counts()}")
print(f"Negative and positive values of validation: \n{val_df['label_n'].value_counts()}")
print(f"Negative and positive values of test: \n{test_df['label_n'].value_counts()}")

In [ ]:
train_df=train_df.reset_index()
val_df=val_df.reset_index()
test_df=test_df.reset_index()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from get_lungs_vh_paper import get_lungs


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation([-15,15]),
        transforms.GaussianBlur(3),
        #transforms.RandomAdjustSharpness(0),
        #transforms.RandomEqualize(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class CovidDataset(Dataset):
    def __init__(self, dataset_df, transform=None):
        self.dataset_df = dataset_df
        self.transform = transform
        
    def __len__(self):
        return self.dataset_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = self.dataset_df['filename'][idx]
        img = Image.open(image_name).convert('RGB')
        label = self.dataset_df['label'][idx]
        
        if self.transform:
            img = self.transform(img)
        return img, label
    
train_dataset = CovidDataset(train_df, data_transforms['train'])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=20)

val_dataset = CovidDataset(val_df, data_transforms['val'])
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=20)

test_dataset = CovidDataset(test_df, data_transforms['val'])
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=20)

In [ ]:
import torchvision
from matplotlib import pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = [30, 20]

def imshow_img(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
# Get a batch of training data
images, labels = next(iter(train_dataloader))

# Make a grid from batch
out = torchvision.utils.make_grid(images)
print(labels)

imshow_img(out)

In [ ]:
from PIL import Image
from scipy import ndimage
import cv2
import numpy as np
import pydicom as dicom
import os

normal_path = '/media/riccelli/Disco 1/datasets_covid/spgc_after_segmentation/Normal'
covid_path = '/media/riccelli/Disco 1/datasets_covid/spgc_after_segmentation/Covid'
cp_path = '/media/riccelli/Disco 1/datasets_covid/spgc_after_segmentation/Pneumonia'

normal_patient_paths = [
    os.path.join(os.getcwd(), normal_path, x)
    for x in os.listdir(normal_path)
]

covid_patient_paths = [
    os.path.join(os.getcwd(), covid_path, x)
    for x in os.listdir(covid_path)
]

cp_patient_paths = [
    os.path.join(os.getcwd(), cp_path, x)
    for x in os.listdir(cp_path)
]

patient_paths = np.concatenate((np.asarray(covid_patient_paths), np.asarray(cp_patient_paths), np.asarray(normal_patient_paths)))

covidctmd_dataset = patient_paths

print("Patients with healthy lung tissue: " + str(len(normal_patient_paths)))

print("Patients with covid symptoms in lung tissue: " + str(len(covid_patient_paths)))
print("Patients with common pneumonia symptoms in lung tissue: " + str(len(cp_patient_paths)))


#class_names = ['Healthy', 'Covid', 'Others']
class_names = ['Pneumonia', 'Covid']
idx_to_class = {i:j for i, j in enumerate(class_names)}
class_to_idx = {value:key for key,value in idx_to_class.items()}


from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import random

class CustomExternalMDDataset(Dataset):
    def __init__(self, paths, transform=None, target_transform=None):
        self.paths = paths
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert('RGB')
        img = transforms.functional.adjust_gamma(img, .8)
        
        label = path.split('/')[-2]
        
        if label == 'Normal':
            label = 'Pneumonia'
            
        label = class_to_idx[label]
        
        
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label
    
covidctmd_dataset = CustomExternalMDDataset(covidctmd_dataset, data_transforms['val'])
covidctmd_dataloader = DataLoader(covidctmd_dataset, batch_size=32, shuffle=False)

In [ ]:
from torchvision import models
from torchvision.models import ResNet50_Weights, SqueezeNet1_0_Weights
import torch.nn as nn


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def create_model(model_name, in_channels, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        #model_ft = models.resnet50(pretrained=use_pretrained)
        model_ft = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG16_bn
        """
        model_ft = models.vgg16_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        #model_ft = models.squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)
        model_ft = models.squeezenet1_0(weights='DEFAULT')
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
        
    elif model_name == "mobilenet":
        """ MobileNet
        """
        model_ft = models.mobilenet_v3_small(weights='DEFAULT')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[3].in_features
        model_ft.classifier[3] = nn.Linear(num_ftrs, num_classes)
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet201(weights='DEFAULT')
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

In [ ]:
import pytorch_lightning as pl
import torch.optim as optim
from sklearn.metrics import classification_report
import torchmetrics
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, mean_absolute_error


class ClassificationModel(pl.LightningModule):
    def __init__(self, model_name, model_hparams, optimizer_name, optimizer_hparams):
        """
        Inputs:
            model_name - Name of the model/CNN to run. Used for creating the model (see function below)
            model_hparams - Hyperparameters for the model, as dictionary.
            optimizer_name - Name of the optimizer to use. Currently supported: Adam, SGD
            optimizer_hparams - Hyperparameters for the optimizer, as dictionary. This includes learning rate, weight decay, etc.
        """
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = create_model(model_name, model_hparams["in_channels"], 
                                  model_hparams["num_classes"], False, use_pretrained=True)
        # Create loss module
        self.loss_module = nn.CrossEntropyLoss()
        
    def forward(self, imgs):
        # Forward function that is run when visualizing the graph
        return self.model(imgs)
    
    def shared_step(self, batch, stage):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
                
        return {
            "loss": loss,
            "labels": labels,
            "preds": preds
        }
    
    def shared_epoch_end(self, outputs, stage):
        labels = torch.cat([x["labels"] for x in outputs]).cpu()
        preds = torch.cat([x["preds"] for x in outputs]).cpu().argmax(dim=-1)
        
        acc = (preds == labels).float().mean()
        
        if stage == 'test':
        
            #print(classification_report(labels, preds, target_names=['Normal', 'Pneumonia', 'Covid']))
            print(classification_report(labels, preds, target_names=['Pneumonia', 'Covid']))

            cm = confusion_matrix(labels, preds)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  #display_labels=['Normal', 'Pneumonia', 'Covid'])
                                  display_labels=['Pneumonia', 'Covid'])
            disp.plot(cmap='Blues')

            plt.show()
            
            print(accuracy_score(labels, preds))
            print(f1_score(labels, preds))
            print(precision_score(labels, preds))
            print(recall_score(labels, preds))
            print(recall_score(labels, preds, pos_label=0))
        
        
        metrics = {
             f"{stage}_acc": acc,
#             f"{stage}_acc_pneumonia": self.acc[1],
#             f"{stage}_acc_covid": self.acc[2],
#             f"{stage}_f1_normal": self.f1[0],
#             f"{stage}_f1_pneumonia": self.f1[1],
#             f"{stage}_f1_covid": self.f1[2],
#             #f"{stage}_precision": accuracy,
#             #f"{stage}_recall": f1_score,
            
         }
        
        self.log_dict(metrics, prog_bar=True)
        
    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, "train")            

    def training_epoch_end(self, outputs):
        return self.shared_epoch_end(outputs, "train")

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, "val")

    def validation_epoch_end(self, outputs):
        return self.shared_epoch_end(outputs, "val")

    def test_step(self, batch, batch_idx):
        return self.shared_step(batch, "test")  

    def test_epoch_end(self, outputs):
        return self.shared_epoch_end(outputs, "test")

    def configure_optimizers(self):
        if self.hparams.optimizer_name == "Adam":
            return optim.AdamW(self.parameters(), **self.hparams.optimizer_hparams)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.loggers import WandbLogger
from pprint import pprint
from random import randint

random_num = randint(0,10000)

model_name = 'densenet'
#model_hparams={"in_channels": 1, "num_classes": 3, "act_fn_name": "relu"}
model_hparams={"in_channels": 1, "num_classes": 2, "act_fn_name": "relu"}
optimizer_name="Adam"
optimizer_hparams={"lr": 1e-4, "weight_decay": 1e-4}

checkpoint_callback = ModelCheckpoint(
                        dirpath='/media/riccelli/Disco 1/datasets_covid/covidxct/weights',    
                        filename=f'covidxct-{model_name}-{random_num}',
                        monitor='val_acc',
                        mode="max",
                        )

#valid per_image_iou
early_stop_callback = EarlyStopping(
                        monitor="val_acc", 
                        patience=5, 
                        verbose=False, 
                        mode="max"
                        )

model = ClassificationModel(model_name, model_hparams, optimizer_name, 
                            optimizer_hparams) 

In [ ]:
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1, 
    precision=16,
    max_epochs=10,
    callbacks = [checkpoint_callback, early_stop_callback],
    accumulate_grad_batches=2,
    )

trainer.fit(
    model, 
    train_dataloaders=train_dataloader, 
    #val_dataloaders=val_dataloader,
    #val_dataloaders=covidctmd_dataloader,
    val_dataloaders=val_dataloader,
    )
        
    #Load best
ckpt_path = f'/media/riccelli/Disco 1/datasets_covid/covidxct/weights/covidxct-{model_name}-{random_num}.ckpt'
loaded_model = ClassificationModel.load_from_checkpoint(ckpt_path)    
    
#     # run test dataset
test_metrics = trainer.test(model, dataloaders=covidctmd_dataloader, verbose=False)
pprint(test_metrics)

test_metrics = trainer.test(loaded_model, dataloaders=covidctmd_dataloader, verbose=False)
pprint(test_metrics)

os.remove(ckpt_path)


In [ ]:
#     # run test dataset
test_metrics = trainer.test(model, dataloaders=covidctmd_dataloader, verbose=False)
pprint(test_metrics)

test_metrics = trainer.test(loaded_model, dataloaders=covidctmd_dataloader, verbose=False)
pprint(test_metrics)

In [ ]:
test_metrics = trainer.test(loaded_model, dataloaders=test_dataloader, verbose=False)
pprint(test_metrics)